## Select the dataset

In [281]:
dataset = "LUNG"

"LUNG"

# Load gene attributes

In [282]:
using CSV
using DataFrames
using Statistics

df = CSV.File("../$dataset/node_attributes.csv", typemap=Dict(Int => Float64)) |> DataFrame    # read the attribute file
df = unique(df, :name)      # remove possible duplicated genes
genenames = df[!, :name]
first(df, 15)

Row,name,label_wo000916,label,transcript_count,gc_content,official_gene_symbol,Gtex_lung,gene_disease_ass_count,oncodb_expression,HPA_lung_match_symbol,mf,bp,cc,biogrid,kegg,reactome,ucsc_tfbs,up_tissue,orth_count,gene_length,Mitochondrion,Nucleus,Endosome,Golgi apparatus,Cytosol,Plasma membrane,Endoplasmic reticulum,Cytoskeleton,Peroxisome,Lysosome,Extracellular space,Vacuole
,String,String3?,String3?,Float64?,Float64?,String15?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?
1,(clone tec14),missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
2,100 kDa coactivator,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
3,14-3-3 tau splice variant,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
4,3-beta-hydroxysteroid dehydrogenase,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
5,3-mercaptopyruvate sulfurtransferase variant,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
6,3'-phosphoadenosine-5'-phosphosulfate synthase,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
7,40S ribosomal protein S15a,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
8,5-aminoimidazole-4-carboxamide ribonucleotide formyltransferase,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
9,5'-3' exoribonuclease,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing


In [283]:
numeric_columns = []
for nm in names(df)
    if Float64 <: eltype(df[!, nm])
        append!(numeric_columns, [nm])
    end
end                                                                   # select only numeric attributes
select!(df, numeric_columns)
#df = df[typeintersect.(colwise(eltype, df), Number) .!= Union{}]      
for col in eachcol(df)                                                # fix missing and naN with mean value
    m = mean(collect(skipmissing(col)))
    std = mean(collect(skipmissing(col)))
    replace!(col, missing => m)
    replace!(col, NaN => m)
    convert.(Float64,col)
end
df = mapcols!(ByRow(Float64), df)                                     # convert Union{missing,Float64} to Float64 types

Row,transcript_count,gc_content,Gtex_lung,gene_disease_ass_count,oncodb_expression,HPA_lung_match_symbol,mf,bp,cc,biogrid,kegg,reactome,ucsc_tfbs,up_tissue,orth_count,gene_length,Mitochondrion,Nucleus,Endosome,Golgi apparatus,Cytosol,Plasma membrane,Endoplasmic reticulum,Cytoskeleton,Peroxisome,Lysosome,Extracellular space,Vacuole
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,8.86255,46.6636,47.1587,8.91382,-0.00984499,29.485,4.55225,8.9346,5.09294,83.3945,4.57453,11.6035,69.4541,4.32568,249.774,70090.1,0.848559,0.963008,0.51042,0.586604,0.921616,0.872525,0.726062,0.850585,0.475802,0.596735,0.667014,0.650805
2,8.86255,46.6636,47.1587,8.91382,-0.00984499,29.485,4.55225,8.9346,5.09294,83.3945,4.57453,11.6035,69.4541,4.32568,249.774,70090.1,0.848559,0.963008,0.51042,0.586604,0.921616,0.872525,0.726062,0.850585,0.475802,0.596735,0.667014,0.650805
3,8.86255,46.6636,47.1587,8.91382,-0.00984499,29.485,4.55225,8.9346,5.09294,83.3945,4.57453,11.6035,69.4541,4.32568,249.774,70090.1,0.848559,0.963008,0.51042,0.586604,0.921616,0.872525,0.726062,0.850585,0.475802,0.596735,0.667014,0.650805
4,8.86255,46.6636,47.1587,8.91382,-0.00984499,29.485,4.55225,8.9346,5.09294,83.3945,4.57453,11.6035,69.4541,4.32568,249.774,70090.1,0.848559,0.963008,0.51042,0.586604,0.921616,0.872525,0.726062,0.850585,0.475802,0.596735,0.667014,0.650805
5,8.86255,46.6636,47.1587,8.91382,-0.00984499,29.485,4.55225,8.9346,5.09294,83.3945,4.57453,11.6035,69.4541,4.32568,249.774,70090.1,0.848559,0.963008,0.51042,0.586604,0.921616,0.872525,0.726062,0.850585,0.475802,0.596735,0.667014,0.650805
6,8.86255,46.6636,47.1587,8.91382,-0.00984499,29.485,4.55225,8.9346,5.09294,83.3945,4.57453,11.6035,69.4541,4.32568,249.774,70090.1,0.848559,0.963008,0.51042,0.586604,0.921616,0.872525,0.726062,0.850585,0.475802,0.596735,0.667014,0.650805
7,8.86255,46.6636,47.1587,8.91382,-0.00984499,29.485,4.55225,8.9346,5.09294,83.3945,4.57453,11.6035,69.4541,4.32568,249.774,70090.1,0.848559,0.963008,0.51042,0.586604,0.921616,0.872525,0.726062,0.850585,0.475802,0.596735,0.667014,0.650805
8,8.86255,46.6636,47.1587,8.91382,-0.00984499,29.485,4.55225,8.9346,5.09294,83.3945,4.57453,11.6035,69.4541,4.32568,249.774,70090.1,0.848559,0.963008,0.51042,0.586604,0.921616,0.872525,0.726062,0.850585,0.475802,0.596735,0.667014,0.650805
9,8.86255,46.6636,47.1587,8.91382,-0.00984499,29.485,4.55225,8.9346,5.09294,83.3945,4.57453,11.6035,69.4541,4.32568,249.774,70090.1,0.848559,0.963008,0.51042,0.586604,0.921616,0.872525,0.726062,0.850585,0.475802,0.596735,0.667014,0.650805


### zscore attributes

In [284]:
using StatsBase
dt = fit(ZScoreTransform, Matrix(df), dims=2)
x = StatsBase.transform(dt, Matrix(df)) #|> DataFrame
df = DataFrame(x, names(df))
df[!, :name] = genenames

19334-element Vector{String}:
 "(clone tec14)"
 "100 kDa coactivator"
 "14-3-3 tau splice variant"
 "3-beta-hydroxysteroid dehydrogenase"
 "3-mercaptopyruvate sulfurtransferase variant"
 "3'-phosphoadenosine-5'-phosphosulfate synthase"
 "40S ribosomal protein S15a"
 "5-aminoimidazole-4-carboxamide ribonucleotide formyltransferase"
 "5'-3' exoribonuclease"
 "6-O-methylguanine-DNA methyltransferase"
 "A0A087WZY1"
 "A0A0G2JLL6"
 "A0A0J9YVX5"
 ⋮
 "ZUP1"
 "ZW10"
 "ZWILCH"
 "ZWINT"
 "ZXDA"
 "ZXDB"
 "ZXDC"
 "ZYG11A"
 "ZYG11B"
 "ZYX"
 "ZZEF1"
 "ZZZ3"

## Load labels

In [285]:
dfl = CSV.File("../$dataset/node_labels.csv", typemap=Dict(Int => Float64)) |> DataFrame    # read the attribute file
dfl = unique(dfl, :name)         # remove gene duplicates
dfl = filter(:most_freq => !ismissing, dfl)

Row,name,ACH-000012,ACH-000015,ACH-000021,ACH-000029,ACH-000030,ACH-000035,ACH-000086,ACH-000092,ACH-000143,ACH-000150,ACH-000153,ACH-000161,ACH-000176,ACH-000187,ACH-000251,ACH-000257,ACH-000261,ACH-000264,ACH-000282,ACH-000290,ACH-000292,ACH-000311,ACH-000319,ACH-000331,ACH-000335,ACH-000337,ACH-000339,ACH-000343,ACH-000355,ACH-000367,ACH-000378,ACH-000390,ACH-000395,ACH-000414,ACH-000416,ACH-000431,ACH-000434,ACH-000438,ACH-000442,ACH-000444,ACH-000448,ACH-000454,ACH-000463,ACH-000481,ACH-000496,ACH-000510,ACH-000511,ACH-000514,ACH-000521,ACH-000525,ACH-000528,ACH-000559,ACH-000562,ACH-000563,ACH-000585,ACH-000587,ACH-000589,ACH-000596,ACH-000627,ACH-000638,ACH-000639,ACH-000645,ACH-000648,ACH-000662,ACH-000665,ACH-000666,ACH-000667,ACH-000677,ACH-000681,ACH-000695,ACH-000698,ACH-000700,ACH-000706,ACH-000718,ACH-000747,ACH-000757,ACH-000766,ACH-000767,ACH-000769,ACH-000774,ACH-000775,ACH-000781,ACH-000785,ACH-000787,ACH-000790,ACH-000791,ACH-000800,ACH-000826,ACH-000837,ACH-000840,ACH-000841,ACH-000843,ACH-000845,ACH-000851,ACH-000853,ACH-000858,ACH-000860,ACH-000861,ACH-000866,⋯
,String15,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,⋯
1,A1BG,aNE,aNE,aNE,aNE,aNE,aNE,NE,aNE,aNE,NE,aNE,aNE,NE,aNE,aNE,aNE,NE,aNE,aNE,aNE,aNE,aNE,aNE,NE,aNE,aNE,aNE,aNE,aNE,aNE,NE,NE,aNE,aNE,aNE,aNE,NE,NE,NE,NE,aNE,aNE,aNE,NE,aNE,aNE,aNE,aNE,NE,aNE,aNE,aNE,aNE,aNE,NE,NE,NE,aNE,aNE,NE,NE,aNE,aNE,NE,NE,aNE,NE,aNE,aNE,NE,NE,NE,NE,NE,NE,NE,NE,aNE,aNE,aNE,aNE,aNE,NE,aNE,aNE,NE,aNE,NE,aNE,NE,NE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,⋯
2,A1CF,NE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,NE,NE,aNE,aNE,NE,aNE,aNE,aNE,NE,NE,aNE,aNE,NE,aNE,aNE,aNE,aNE,aNE,aNE,NE,NE,aNE,aNE,aNE,aNE,aNE,aNE,NE,aNE,NE,NE,aNE,aNE,NE,aNE,aNE,aNE,NE,aNE,aNE,aNE,NE,aNE,NE,aNE,aNE,aNE,NE,aNE,aNE,aNE,aNE,NE,aNE,NE,aNE,aNE,aNE,aNE,aNE,NE,NE,NE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,NE,aNE,aNE,NE,NE,aNE,aNE,aNE,NE,aNE,aNE,aNE,aNE,NE,aNE,NE,aNE,aNE,⋯
3,A2M,NE,aNE,aNE,NE,NE,NE,NE,NE,NE,aNE,aNE,NE,aNE,aNE,aNE,NE,NE,aNE,aNE,NE,aNE,aNE,NE,NE,aNE,aNE,NE,NE,aNE,NE,aNE,NE,NE,NE,NE,aNE,aNE,NE,aNE,aNE,NE,NE,NE,NE,NE,NE,NE,NE,aNE,NE,aNE,NE,NE,NE,NE,aNE,aNE,NE,NE,aNE,NE,NE,NE,aNE,NE,NE,NE,aNE,aNE,NE,NE,NE,NE,NE,aNE,aNE,NE,aNE,aNE,aNE,NE,aNE,aNE,NE,aNE,aNE,aNE,NE,aNE,NE,aNE,aNE,NE,NE,NE,NE,NE,NE,NE,⋯
4,A2ML1,NE,NE,aNE,NE,NE,NE,NE,NE,aNE,NE,NE,NE,NE,NE,NE,aNE,NE,NE,aNE,NE,NE,NE,NE,NE,aNE,aNE,NE,NE,aNE,aNE,NE,NE,NE,aNE,NE,NE,aNE,NE,NE,NE,NE,NE,NE,NE,NE,NE,NE,NE,NE,NE,NE,NE,NE,aNE,NE,NE,NE,aNE,aNE,NE,NE,NE,NE,aNE,NE,aNE,NE,aNE,aNE,NE,NE,NE,NE,NE,NE,aNE,NE,aNE,aNE,NE,NE,NE,NE,NE,NE,NE,NE,NE,NE,aNE,NE,NE,NE,NE,NE,NE,aNE,aNE,aNE,⋯
5,A3GALT2,NE,aNE,NE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,NE,aNE,aNE,NE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,NE,aNE,NE,aNE,NE,aNE,aNE,aNE,aNE,NE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,NE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,NE,aNE,aNE,aNE,NE,NE,NE,aNE,aNE,NE,aNE,NE,aNE,aNE,aNE,aNE,NE,aNE,aNE,NE,aNE,NE,NE,NE,NE,aNE,aNE,aNE,NE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,⋯
6,A4GALT,aNE,aNE,aNE,aNE,aNE,NE,NE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,NE,aNE,aNE,aNE,aNE,aNE,NE,aNE,aNE,NE,aNE,aNE,aNE,NE,aNE,NE,aNE,NE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,NE,NE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,NE,aNE,aNE,NE,aNE,aNE,aNE,aNE,NE,NE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE

## Select labels

In [286]:
dfl = filter(row -> row.most_freq ∈ ["E","NE"], dfl)

Row,name,ACH-000012,ACH-000015,ACH-000021,ACH-000029,ACH-000030,ACH-000035,ACH-000086,ACH-000092,ACH-000143,ACH-000150,ACH-000153,ACH-000161,ACH-000176,ACH-000187,ACH-000251,ACH-000257,ACH-000261,ACH-000264,ACH-000282,ACH-000290,ACH-000292,ACH-000311,ACH-000319,ACH-000331,ACH-000335,ACH-000337,ACH-000339,ACH-000343,ACH-000355,ACH-000367,ACH-000378,ACH-000390,ACH-000395,ACH-000414,ACH-000416,ACH-000431,ACH-000434,ACH-000438,ACH-000442,ACH-000444,ACH-000448,ACH-000454,ACH-000463,ACH-000481,ACH-000496,ACH-000510,ACH-000511,ACH-000514,ACH-000521,ACH-000525,ACH-000528,ACH-000559,ACH-000562,ACH-000563,ACH-000585,ACH-000587,ACH-000589,ACH-000596,ACH-000627,ACH-000638,ACH-000639,ACH-000645,ACH-000648,ACH-000662,ACH-000665,ACH-000666,ACH-000667,ACH-000677,ACH-000681,ACH-000695,ACH-000698,ACH-000700,ACH-000706,ACH-000718,ACH-000747,ACH-000757,ACH-000766,ACH-000767,ACH-000769,ACH-000774,ACH-000775,ACH-000781,ACH-000785,ACH-000787,ACH-000790,ACH-000791,ACH-000800,ACH-000826,ACH-000837,ACH-000840,ACH-000841,ACH-000843,ACH-000845,ACH-000851,ACH-000853,ACH-000858,ACH-000860,ACH-000861,ACH-000866,⋯
,String15,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,⋯
1,A2M,NE,aNE,aNE,NE,NE,NE,NE,NE,NE,aNE,aNE,NE,aNE,aNE,aNE,NE,NE,aNE,aNE,NE,aNE,aNE,NE,NE,aNE,aNE,NE,NE,aNE,NE,aNE,NE,NE,NE,NE,aNE,aNE,NE,aNE,aNE,NE,NE,NE,NE,NE,NE,NE,NE,aNE,NE,aNE,NE,NE,NE,NE,aNE,aNE,NE,NE,aNE,NE,NE,NE,aNE,NE,NE,NE,aNE,aNE,NE,NE,NE,NE,NE,aNE,aNE,NE,aNE,aNE,aNE,NE,aNE,aNE,NE,aNE,aNE,aNE,NE,aNE,NE,aNE,aNE,NE,NE,NE,NE,NE,NE,NE,⋯
2,A2ML1,NE,NE,aNE,NE,NE,NE,NE,NE,aNE,NE,NE,NE,NE,NE,NE,aNE,NE,NE,aNE,NE,NE,NE,NE,NE,aNE,aNE,NE,NE,aNE,aNE,NE,NE,NE,aNE,NE,NE,aNE,NE,NE,NE,NE,NE,NE,NE,NE,NE,NE,NE,NE,NE,NE,NE,NE,aNE,NE,NE,NE,aNE,aNE,NE,NE,NE,NE,aNE,NE,aNE,NE,aNE,aNE,NE,NE,NE,NE,NE,NE,aNE,NE,aNE,aNE,NE,NE,NE,NE,NE,NE,NE,NE,NE,NE,aNE,NE,NE,NE,NE,NE,NE,aNE,aNE,aNE,⋯
3,AADAC,NE,NE,aNE,aNE,NE,aNE,aNE,aNE,NE,aNE,NE,NE,NE,aNE,aNE,aNE,aNE,NE,aNE,NE,aNE,NE,NE,NE,NE,aNE,NE,NE,aNE,NE,NE,NE,NE,NE,NE,NE,NE,NE,NE,aNE,NE,NE,NE,NE,aNE,NE,NE,NE,aNE,NE,NE,aNE,NE,NE,NE,NE,NE,NE,aNE,NE,NE,NE,NE,aNE,aNE,NE,aNE,NE,NE,NE,NE,aNE,NE,NE,NE,NE,NE,aNE,NE,aNE,NE,NE,aNE,NE,NE,NE,NE,NE,NE,aNE,aNE,aNE,aNE,NE,NE,NE,NE,NE,NE,⋯
4,AADACL4,NE,NE,NE,NE,NE,aNE,NE,aNE,aNE,aNE,aNE,NE,NE,NE,NE,NE,NE,aNE,NE,NE,aNE,NE,NE,NE,aNE,NE,NE,aNE,NE,NE,NE,NE,NE,NE,aNE,aNE,NE,NE,NE,aNE,NE,NE,NE,NE,NE,NE,aNE,NE,aNE,NE,aNE,aNE,aNE,NE,NE,NE,NE,aNE,NE,NE,NE,NE,NE,NE,NE,NE,NE,aNE,NE,NE,NE,aNE,NE,NE,aNE,NE,aNE,NE,NE,NE,NE,NE,NE,NE,aNE,NE,NE,NE,NE,NE,NE,NE,NE,aNE,NE,NE,NE,NE,NE,⋯
5,AAGAB,aNE,NE,aNE,aNE,NE,aNE,aNE,NE,NE,NE,aNE,NE,NE,aNE,aNE,NE,NE,aNE,NE,NE,NE,NE,aNE,aNE,aNE,aNE,aNE,aNE,NE,NE,aNE,NE,NE,NE,aNE,NE,NE,NE,NE,aNE,aNE,aNE,aNE,aNE,NE,NE,aNE,NE,NE,NE,NE,NE,NE,aNE,NE,NE,aNE,aNE,aNE,aNE,aNE,NE,NE,NE,NE,aNE,aNE,NE,NE,NE,aNE,aNE,NE,NE,aNE,NE,aNE,aNE,aNE,aNE,aNE,NE,NE,aNE,aNE,aNE,NE,NE,aNE,NE,aNE,aNE,aNE,aNE,aNE,NE,aNE,NE,aNE,⋯
6,AARS1,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,⋯
7,AASS,NE,aNE,aNE,NE,NE,aNE,NE,aNE,NE,NE,NE,NE,NE,NE,aNE,aNE,aNE,aNE,aNE,NE,NE,NE,NE,NE,NE,NE,NE,NE,aNE,NE,NE,NE,aNE,NE,NE,aNE,NE,aNE,NE,aNE,NE,NE,aNE,aNE,NE,NE,NE,NE,aNE,NE,NE,aNE,NE,a

In [287]:
genenames = dfl[!, :name]
dfall = innerjoin(dfl,df; on=:name)[!, Not(r"ACH-")]         # join labels and attributes dataframes

Row,name,most_freq,transcript_count,gc_content,Gtex_lung,gene_disease_ass_count,oncodb_expression,HPA_lung_match_symbol,mf,bp,cc,biogrid,kegg,reactome,ucsc_tfbs,up_tissue,orth_count,gene_length,Mitochondrion,Nucleus,Endosome,Golgi apparatus,Cytosol,Plasma membrane,Endoplasmic reticulum,Cytoskeleton,Peroxisome,Lysosome,Extracellular space,Vacuole
,String15,String3,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,A2M,NE,-0.208327,-0.205689,0.167256,-0.206908,-0.21007,-0.0609493,-0.208217,-0.208436,-0.209199,-0.192072,-0.209636,-0.208108,-0.20309,-0.208981,-0.200036,5.08809,-0.209636,-0.209636,-0.209636,-0.209636,-0.209636,-0.209636,-0.209636,-0.209636,-0.209636,-0.209636,-0.209636,-0.209636
2,A2ML1,NE,-0.189503,-0.186607,-0.190233,-0.189668,-0.189936,-0.190235,-0.189914,-0.190079,-0.189996,-0.180296,-0.189867,-0.189289,-0.187695,-0.189914,-0.177172,5.1025,-0.190161,-0.190161,-0.190243,-0.190161,-0.190161,-0.190161,-0.190161,-0.190161,-0.190243,-0.190161,-0.190161,-0.190161
3,AADAC,NE,-0.192001,-0.17977,-0.192144,-0.192369,-0.194029,-0.189394,-0.190165,-0.191634,-0.192001,-0.192369,-0.191056,-0.191634,-0.187961,-0.191634,-0.12515,5.1021,-0.192369,-0.192369,-0.192736,-0.192369,-0.192369,-0.192369,-0.192369,-0.192369,-0.192369,-0.192369,-0.192369,-0.192369
4,AAGAB,NE,-0.190414,-0.187262,-0.188831,-0.190705,-0.191028,-0.18957,-0.190997,-0.190997,-0.190802,-0.187987,-0.19065,-0.189967,-0.182939,-0.190414,-0.157602,5.10242,-0.190997,-0.190997,-0.190997,-0.190997,-0.190997,-0.190997,-0.190997,-0.191094,-0.191094,-0.190997,-0.191094,-0.190997
5,AARS1,E,-0.186829,-0.185765,-0.186105,-0.192443,-0.192725,-0.188681,-0.191461,-0.191601,-0.192022,-0.177565,-0.192583,-0.192162,-0.181776,-0.191882,-0.147246,5.10231,-0.192605,-0.192589,-0.192652,-0.192641,-0.192594,-0.192601,-0.192622,-0.192604,-0.192657,-0.19264,-0.19263,-0.192632
6,AASS,NE,-0.188684,-0.188051,-0.188579,-0.189547,-0.190477,-0.190185,-0.189879,-0.190145,-0.190078,-0.186958,-0.190344,-0.190278,-0.185697,-0.19041,-0.167904,5.10247,-0.19041,-0.19041,-0.190477,-0.190477,-0.19041,-0.19041,-0.19041,-0.19041,-0.19041,-0.190477,-0.190477,-0.190477
7,ABCA7,NE,-0.18909,-0.180284,-0.186932,-0.190746,-0.192715,-0.192092,-0.19116,-0.187019,-0.190332,-0.188676,-0.19261,-0.192196,-0.174804,-0.191781,-0.15058,5.10233,-0.19261,-0.19261,-0.19261,-0.19261,-0.19261,-0.19261,-0.19261,-0.19261,-0.19261,-0.19261,-0.19261,-0.19261
8,ABCA9,NE,-0.189095,-0.187279,-0.188673,-0.189463,-0.189698,-0.1892,-0.18934,-0.189463,-0.189463,-0.189095,-0.189524,-0.189402,-0.18579,-0.18934,-0.183403,5.10252,-0.189524,-0.189524,-0.189524,-0.189524,-0.189524,-0.189524,-0.189524,-0.189524,-0.189524,-0.189524,-0.189524,-0.189524
9,ABCB7,E,-0.18943,-0.188478,-0.18931,-0.18983,-0.19027,-0.18992,-0.189981,-0.18983,-0.190131,-0.18332,-0.190231,-0.190031,-0.185523,-0.190081,-0.173354,5.10249,-0.190231,-0.190231,-0.190231,-0.190231,-0.190231,-0.190231,-0.190231,-0.190231,-0.190231,-0.190231,-0.190231,-0.190231


## Load embedding

In [288]:
dfe = CSV.File("../$dataset/embeddings/PPI_Node2Vec_64.csv") |> DataFrame    # read the attribute file
dfall = innerjoin(dfall,dfe; on=:name)               # join labels/attributes dataframe with embedding

Row,name,most_freq,transcript_count,gc_content,Gtex_lung,gene_disease_ass_count,oncodb_expression,HPA_lung_match_symbol,mf,bp,cc,biogrid,kegg,reactome,ucsc_tfbs,up_tissue,orth_count,gene_length,Mitochondrion,Nucleus,Endosome,Golgi apparatus,Cytosol,Plasma membrane,Endoplasmic reticulum,Cytoskeleton,Peroxisome,Lysosome,Extracellular space,Vacuole,Node2Vec_1,Node2Vec_2,Node2Vec_3,Node2Vec_4,Node2Vec_5,Node2Vec_6,Node2Vec_7,Node2Vec_8,Node2Vec_9,Node2Vec_10,Node2Vec_11,Node2Vec_12,Node2Vec_13,Node2Vec_14,Node2Vec_15,Node2Vec_16,Node2Vec_17,Node2Vec_18,Node2Vec_19,Node2Vec_20,Node2Vec_21,Node2Vec_22,Node2Vec_23,Node2Vec_24,Node2Vec_25,Node2Vec_26,Node2Vec_27,Node2Vec_28,Node2Vec_29,Node2Vec_30,Node2Vec_31,Node2Vec_32,Node2Vec_33,Node2Vec_34,Node2Vec_35,Node2Vec_36,Node2Vec_37,Node2Vec_38,Node2Vec_39,Node2Vec_40,Node2Vec_41,Node2Vec_42,Node2Vec_43,Node2Vec_44,Node2Vec_45,Node2Vec_46,Node2Vec_47,Node2Vec_48,Node2Vec_49,Node2Vec_50,Node2Vec_51,Node2Vec_52,Node2Vec_53,Node2Vec_54,Node2Vec_55,Node2Vec_56,Node2Vec_57,Node2Vec_58,Node2Vec_59,Node2Vec_60,Node2Vec_61,Node2Vec_62,Node2Vec_63,Node2Vec_64
,String15,String3,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,MAP2K4,NE,-0.189375,-0.188425,-0.189193,-0.189505,-0.190021,-0.189513,-0.189548,-0.18916,-0.189806,-0.187482,-0.189117,-0.188472,-0.186794,-0.189763,-0.176596,5.1025,-0.189978,-0.189978,-0.189978,-0.189978,-0.189978,-0.189978,-0.189978,-0.189978,-0.189978,-0.189978,-0.189978,-0.189978,-0.880204,0.116223,1.63286,-0.0199882,-1.34703,0.310535,-1.22341,0.733396,0.193555,0.568226,0.760626,-0.624368,0.978965,0.992708,-0.292229,-0.145836,-0.390245,1.00771,0.239802,-0.196372,0.799632,-0.246882,-0.482066,-0.38869,-0.274809,0.523403,-0.09583,-1.02993,-0.52841,0.00952045,0.277327,-1.29056,0.200026,-0.454238,0.0585348,0.787128,-0.506243,0.423881,-0.192358,1.33055,-0.550193,-0.53506,-0.16216,-1.4428,0.388072,-0.00178415,-0.397996,0.262327,-0.803593,0.164821,-0.0477091,-0.546508,1.09682,0.440914,-0.475373,1.523,-1.06317,1.60738,1.20912,-0.746353,-0.706868,1.26847,0.559755,0.711739
2,FNTA,E,-0.18989,-0.186958,-0.188557,-0.190412,-0.191278,-0.187736,-0.190096,-0.190506,-0.190711,-0.183324,-0.191224,-0.18948,-0.188864,-0.190609,-0.158083,5.10242,-0.191224,-0.191224,-0.191224,-0.191224,-0.191224,-0.191224,-0.191224,-0.191224,-0.191224,-0.191224,-0.191224,-0.191224,-0.220919,-0.450426,-0.392567,-0.387502,-0.0979409,0.611785,-0.305979,-0.284229,0.171214,0.408684,-0.668236,0.340381,-1.03289,0.73598,-0.786434,0.348738,0.660534,-0.159548,1.24158,0.454414,-0.574716,0.144483,0.590627,0.615918,-0.250537,-1.65547e-5,0.174203,0.777047,-1.13016,0.568627,-1.53166,-0.0985904,0.55495,-0.960184,-0.17093,0.285542,-0.757103,-1.03306,-0.0228431,-0.741596,-1.6944,0.629666,-0.984128,-0.733231,1.47732,-0.636289,1.50903,0.0936772,-0.283757,1.46552,-0.531345,-0.65435,0.360331,0.743414,0.451645,1.98642,0.0600894,-1.11211,0.426009,0.000679517,1.02748,0.826205,0.567026,-0.30442
3,ACVR1,NE,-0.189066,-0.188521,-0.189249,-0.188991,-0.190036,-0.18938,-0.189406,-0.188159,-0.189746,-0.185401,-0.189897,-0.18961,-0.18457,-0.189973,-0.180338,5.10251,-0.190011,-0.190011,-0.190011,-0.190011,-0.190011,-0.190011,-0.190011,-0.190011,-0.190048,-0.190011,-0.190011,-0.190011,-0.127996,-0.833414,0.6089,-0.632251,0.583655,0.257014,-0.00226043,0.312416,0.586801,0

In [289]:
X = Matrix(dfall[!, Not(r"name|most_freq")])
y = vec(Matrix(DataFrames.select(dfall, [:most_freq])))
typeof(X), size(X), typeof(y), size(y)

(Matrix{Float64}, (5904, 92), Vector{String3}, (5904,))

## Classification with LightGBM

In [296]:
using MLJ
using MLJBase
using LightGBM
using StableRNGs
rng = StableRNG(566)

LIGHTGBM_SOURCE = abspath("~/LightGBM-3.2.0")
y = CategoricalArray(y)
rng = StableRNG(566)
model = LightGBM.MLJInterface.LGBMClassifier(
    objective = "multiclass", 
    num_iterations = 100,
    learning_rate = .1, 
    early_stopping_round = 15,
    num_leaves = 1000,
    #num_class = 2,
    #metric = ["matthews_correlation"]
)
#model = DecisionTree.RandomForestClassifier()
evalmode = "cv"

preds = []
targets = []
if evalmode == "evaluate"
    cv=StratifiedCV(nfolds=5, shuffle=true, rng=rng)
    res = evaluate(model, X, y, resampling=cv, measure=[accuracy, balanced_accuracy, matthews_correlation])
    display(res)
elseif evalmode == "cv"
    stratified_cv = StratifiedCV(nfolds=5, rng=1234)
    rows = 1:size(X)[1]
    splits = MLJBase.train_test_pairs(stratified_cv, rows, y)
    for s in splits
        train, test = s[1], s[2]
        mach = machine(model, X, y, scitype_check_level=0)
        MLJ.fit!(mach, rows=train, verbosity=-1) 
        targets = cat(targets, vec(y[test, :]), dims=1)
        preds = cat(preds, MLJ.predict_mode(mach, rows=test), dims=1)
    end
    ŷ = convert(CategoricalArray{String3,1,UInt32},  preds)
    y_targets = convert(CategoricalArray{String3,1,UInt32}, targets)
    println("Accuracy: \t$(MLJBase.Accuracy()(ŷ,y_targets))")
    println("Balanced Acc: \t$(MLJBase.BalancedAccuracy()(ŷ,y_targets))")
    println("MCC: \t\t$(MLJBase.MatthewsCorrelation()(ŷ,y_targets))")
    confusionmatrix(ŷ,y_targets)
elseif evalmode == "split"
    train, test = partition(eachindex(y), 0.8, shuffle=true, rng=rng)
    mach = machine(model, X, y, scitype_check_level=0)
    MLJ.fit!(mach, rows=train, verbosity=-1) 
    ŷ = MLJ.predict_mode(mach, rows=test)
    y_targets = vec(y[test,:])
    println("Accuracy: \t$(MLJBase.Accuracy()(ŷ,y_targets))")
    println("Balanced Acc: \t$(MLJBase.BalancedAccuracy()(ŷ,y_targets))")
    println("MCC: \t\t$(MLJBase.MatthewsCorrelation()(ŷ,y_targets))")
    confusionmatrix(ŷ,y_targets)
end

Accuracy: 	0.9454607046070461
Balanced Acc: 	0.8485735678890823
MCC: 		0.7641672486287469


Row,NE,E,name
,Int64,Int64,String3
1,4979,79,NE
2,243,603,E


## Classification with RandomForest

In [291]:
using ScikitLearn.CrossValidation: cross_val_predict
using DecisionTree
using MLJBase
using MLJ
using CategoricalArrays

using LightGBM
LIGHTGBM_SOURCE = abspath("~/LightGBM-3.2.0")

function confusionmatrix(predictions, labels)
   classes = vec(unique(labels))
   d = size(classes)[1]
   idx = Dict(zip(classes,Vector(1:d)))
   c = zeros(Int64, d,d)
   for i in 1:size(labels)[1]
       c[idx[labels[i]] ,idx[predictions[i]]] += 1 
   end 
   df = DataFrame(c, classes)
   df[!, :name] = classes
   return df
end
y_targets = convert(CategoricalArray{String3,1,UInt32}, y)
model = DecisionTree.RandomForestClassifier()
ŷ = convert(CategoricalArray{String3,1,UInt32}, cross_val_predict(model, X, y; cv=5))
println("Accuracy: \t$(MLJBase.Accuracy()(ŷ,y_targets))")
println("Balanced Acc: \t$(MLJBase.BalancedAccuracy()(ŷ,y_targets))")
println("MCC: \t\t$(MLJBase.MatthewsCorrelation()(ŷ,y_targets))")
confusionmatrix(ŷ,y)

Accuracy: 	0.9202235772357723
Balanced Acc: 	0.7865967542464852
MCC: 		0.6464370511575377


Row,NE,E,name
,Int64,Int64,String3
1,4926,132,NE
2,339,507,E
